In [1]:
import numpy as np
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [2]:
training_file = '2600-0.txt'

In [3]:
raw_text = open(training_file, 'r', encoding='utf-8').read()

In [4]:
raw_text = raw_text.lower()

In [5]:
print(raw_text[:200])

the project gutenberg ebook of war and peace, by leo tolstoy

this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restric


In [6]:
nChars = len(raw_text)

In [7]:
chars = sorted(list(set(raw_text)))

In [8]:
nVocabs = len(chars)

In [9]:
print("The total number of vocabularies is: ", nVocabs)

The total number of vocabularies is:  83


In [10]:
allWords = raw_text.split()

In [11]:
uniqueWords = sorted(list(set(allWords)))

In [12]:
indexToChars = dict((i, c) for i, c in enumerate(chars))
charsToIndex = dict((c, i) for i, c in enumerate(chars))

In [13]:
import numpy as np

In [14]:
seqLength = 160

In [15]:
nSeq = int(np.floor((nChars - 1) / seqLength))

In [16]:
print("The total number of sequences is: ", nSeq)

The total number of sequences is:  20171


In [17]:
X = np.zeros((nSeq, seqLength, nVocabs))
y = np.zeros((nSeq, seqLength, nVocabs))

In [18]:
for i in range(nSeq):

    xSequence = raw_text[i * seqLength: (i + 1) * seqLength]

    xSequenceIndex = [charsToIndex[char] for char in xSequence]

    inputSequence = np.zeros((seqLength, nVocabs))

    for j in range(seqLength):

        inputSequence[j][xSequenceIndex[j]] = 1

    X[i] = inputSequence

    ySequence = raw_text[i * seqLength + 1: (i + 1) * seqLength + 1]

    ySequenceIndex = [charsToIndex[char] for char in ySequence]

    targetSequence = np.zeros((seqLength, nVocabs))

    for j in range(seqLength):

        targetSequence[j][ySequenceIndex[j]] = 1

    y[i] = targetSequence

In [19]:
tensorflow.random.set_seed(42)

In [20]:
hiddenUnits = 70
dropOut = 0.4

In [21]:
batchSize = 100
nEpoch = 20

In [22]:
model = Sequential()

In [23]:
model.add(LSTM(hiddenUnits, input_shape=(None, nVocabs), return_sequences=True, dropout=dropOut))

In [24]:
model.add(LSTM(hiddenUnits, return_sequences=True, dropout=dropOut))

In [25]:
model.add(Dense(nVocabs, activation='softmax'))

In [26]:
optimizer = keras.optimizers.Adam(lr=0.001)

c:\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [27]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [28]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 70)          43120     
                                                                 
 lstm_1 (LSTM)               (None, None, 70)          39480     
                                                                 
 dense (Dense)               (None, None, 83)          5893      
                                                                 
Total params: 88,493
Trainable params: 88,493
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping

In [30]:
filePath = 'model.hdf5'

In [31]:
checkpoint = ModelCheckpoint(filePath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [32]:
earlyStop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=3)

In [33]:
import numpy as np

In [34]:
def generateText(model, genLength, nVocab, indexToChar):
    index = np.random.randint(nVocab)
    yChar = [indexToChar[index]]
    x = np.zeros((1, genLength, nVocab))

    for i in range(genLength):
        x[0, i, :][index] = 1
        pred = model.predict(x[:, :i+1, :])[0][-1]
        index = np.argmax(pred)
        yChar.append(indexToChar[index])

    return ''.join(yChar)

In [35]:
class ResultChecker(Callback):
    def __init__(self, model, genLength, nVocab, indexToChar):
        self.model = model
        self.genLength = genLength
        self.nVocab = nVocab
        self.indexToChar = indexToChar

    def on_epoch_end(self, epoch, logs={}):
        print(generateText(self.model, self.genLength, self.nVocab, self.indexToChar))

In [36]:
resultChecker = ResultChecker(model, 100, nVocabs, indexToChars)

In [ ]:
model.fit(X, y, batch_size=batchSize, epochs=nEpoch, callbacks=[checkpoint, earlyStop, resultChecker])